<a href="https://colab.research.google.com/github/Oleksii-Adamov/AES-md5/blob/main/AES_and_md5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
!pip install bitarray
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import hashlib
import math
from bitarray import bitarray
from Crypto import Cipher
import binascii
import os
import copy

#MD5

MD5 - алгоритм хешування, який перетворює повідомлення(текст) будь-якої довжини у 128-бітне хеш представлення.
Алгоритм полягає у таких кроках:
1. Вирівнювання. До вхідної послідовністі бітів додається одиничний біт та нульові біти, поки загальна кількість бітів не стане дорівнювати 448 по модулю 512, причому біти додаються навіть якщо довжина повідомлення у бітах була кратна 512. Після цього додається 64 наймолодших біта бітового представлення довжини повідомлення до вирівнювання. Таким чином довжина повідомлення у бітах становиться кратною 512. 
2. Вводяться 32-бітні змінні-числа A,B,C,D з наперед заданими значеннями, та нелінійні функції:

  $F(B,C,D)=(B ∧ C) \vee (\neg B ∧ D)$,

  $G(B,C,D)=(B ∧ D) \vee ( C ∧ \neg D)$,

  $H(B,C,D)=B ⊕ C ⊕ D$,

  $I(B,C,D) = C ⊕ (B \vee \neg D)$.
3. Оброблюємо вхідне повідомлення шматками по 512 бітів. Для кожного 512-бітного шматка: 
 1. Ініціалізуємо AA=A,BB=B,CC=C,DD=D.
 2. Виконуємо 4 раунди по 16 однотипних операцій ($0\leq i<4*16$):

    $g = i$, якщо $i < 16$; $g = (5 * i + 1) % 16$, якщо $16 \leq i < 32$; $g = (3 * i + 5) % 16$, якщо $32 \leq i < 48$, $g = (7 * i) % 16$, якщо $48 <= i < 64$; 

    $f = (F_r(BB,CC,DD)+K[i]+M[g])<<<s[i]$, $F_r$ - це функція $F$ у першому раунді, $G$ - у другому, і так далі; $K$ - масив констант, $K[i] = floor(2^{32}*abs(sin(i+1)))$; $M[g] - g$-ий 32-бітний шматок опрацьовуємого 512-бітного шматка; $<<<$ - циклічний побітовий ссув вліво, $s$ - масив констант, які вказують на яку кількість бітів робити ссув;

    AA = DD;

    DD = CC;

    CC = BB;

    BB += $f$.
 3. Оновлюємо змінні A += AA, B += BB, C += CC, D += DD.
 Додавання виконується по модулю 2^32.
4. Результуючий 128-бітний хеш є конкатенацією 32-бітних представлень змінних A, B, C, D (менш значуші біти першими).

In [ ]:
circular_bitshift = lambda x, n: (x << n) | (x >> (32 - n))
add_modulo_2_in_32 = lambda a, b: (a + b) % pow(2, 32)

class Md5:
    def __init__(self):
        self.bits = None
        self.text_in_bytes = None
        self.s = [7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22,
         5, 9, 14, 20, 5, 9, 14, 20, 5, 9, 14, 20, 5, 9, 14, 20,
         4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23,
         6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21]
        self.K = [math.floor(pow(2, 32) * abs(math.sin(i + 1))) for i in range(64)]
    
    def _padding(self):
        self.bits.append(1)
        while len(self.bits) % 512 != 448:
            self.bits.append(0)
        length_modulo = (len(self.text_in_bytes) * 8)
        length_modulo_in_bytes = length_modulo.to_bytes(8, "little")
        length_modulo_in_bits = bitarray(endian = "big")
        length_modulo_in_bits.frombytes(length_modulo_in_bytes)
        self.bits.extend(length_modulo_in_bits)
    
    def hash(self, text_in_bytes):
        self.text_in_bytes = text_in_bytes
        self.bits = bitarray()
        self.bits.frombytes(text_in_bytes)
        self._padding()
        F = lambda b, c, d: (b & c) | (~b & d)
        G = lambda b, c, d: (d & b) | (~d & c)
        H = lambda b, c, d: b ^ c ^ d
        I = lambda b, c, d: c ^ (b | ~d)
        A = 0x67452301
        B = 0xefcdab89
        C = 0x98badcfe
        D = 0x10325476
        number_of_512bit_chunks = len(self.bits) // 512
        for chunk_index in range(0, number_of_512bit_chunks):
            start_index = chunk_index * 512
            chunks_of_32bits = [self.bits[start_index + (x * 32) : start_index + (x * 32) + 32] for x in range(0, 16)]
            chunks_of_32bits_as_integers = [int.from_bytes(chunk_of_32bits.tobytes(), byteorder="little") for chunk_of_32bits in chunks_of_32bits]

            AA = A
            BB = B
            CC = C
            DD = D

            for i in range(4 * 16):
                if i < 16:
                    g = i
                    function_result = F(BB, CC, DD)
                elif i < 32:
                    g = (5 * i + 1) % 16
                    function_result = G(BB, CC, DD)
                elif i < 48:
                    g = (3 * i + 5) % 16
                    function_result = H(BB, CC, DD)
                else:
                    g = (7 * i) % 16
                    function_result = I(BB, CC, DD)
                
                function_result = circular_bitshift(add_modulo_2_in_32(add_modulo_2_in_32(add_modulo_2_in_32(function_result, AA), self.K[i]), chunks_of_32bits_as_integers[g]), self.s[i])

                AA = DD
                DD = CC
                CC = BB
                BB = add_modulo_2_in_32(BB, function_result)

            A = add_modulo_2_in_32(A, AA)
            B = add_modulo_2_in_32(B, BB)
            C = add_modulo_2_in_32(C, CC)
            D = add_modulo_2_in_32(D, DD)

        return b''.join([A.to_bytes(4, "little"), B.to_bytes(4, "little"), C.to_bytes(4, "little"), D.to_bytes(4, "little")])

In [ ]:
md5 = Md5()
md5.hash(b"password").hex()

'5f4dcc3b5aa765d61d8327deb882cf99'

Звіримо з бібліотечним MD5

In [ ]:
hashlib.md5(b"password").hexdigest()

'5f4dcc3b5aa765d61d8327deb882cf99'

#AES

Алгоритм AES шифрує блоки по 128 біт, за допомогою 128, 192, або 256-бітного ключа. Будемо використовувати 128-бітний ключ.
Спочатку відбувається розгортання ключа, за допомогою якого з 128-бітного ключа $K$ отримується $R+1$ 128-бітних ключів ($R=10$(кількість раундів)), кожний з яких складається з 32-бітних слів $W_i,0 \leq i \leq 4R-1$ які отримаються за формулою:

 $W_i$ = $K_i$, якщо $i < N$;

 $W_i$ = $W_{i-N}\oplus SubWord(RotWord(W_{i-1}))\oplus rcon_{i/N}$, якщо $i \geq N$ і $i\equiv 0(mod N)$;

 $W_i$ = $W_{i-N}\oplus SubWord(W_{i-1})$, якщо якщо $i \geq N$, $N\geq 6$ і $i\equiv 4(mod N)$;

 $W_i$ = $W_{i-N}\oplus W_{i-1}$, інакше.

 Де $N$ - кількість 32-бітних слів у ключі, $RotWord$ - циклічний ссув на один біт уліво, $SubWord$ - застосування S перетворення до кожного з байтів, $rcon_{i}$ - 32-бітна констатна для $i$-го раунду $rcon_{i} = [rc_i \quad 00_{16} \quad 00_{16} \quad 00_{16}]$.

 Шифрування 128-бітного блока полягає у наступному:
 1. По стовпчиках формується матриця 4x4 з байтів блоку.
 2. $AddRoundKey$ (береться ключ 0-го раунду)
 3. $R-1$ раундів операцій:

    1. $SubBytes$

    2. $ShiftRows$

    3. $MixColumns$

    4. $AddRoundKey$
 4. Останній раунд, який складається з операцій:
    1. $SubBytes$
    2. $ShiftRows$
    3. $AddRoundKey$

 Операція $AddRoundKey$ полягає у xor-і стану-матриці та ключа для цього раунду.

 Операція $SubBytes$ полягає у застосуванні S перетворення до кожного елементу матриці, це перетворення можна задати константним масивом.

 Операція $ShiftRows$ циклічно сдвигає байти у рядках матриці вліво, на 0 дле першого рядка, на 1 для другого, на 2 для третього та на 3 для четвертого рядка.

 Операція $MixColumns$ полягає у множенні кожного стовпчика матриці на певну константну матрицю зліва, причому операцію додавання замінює xor, а для операції множення біти трактуються як коефіцієнти полінома 7-го степеня, ці поліноми множаться по модулю незвідного полінома $x^8 + x^4 + x^3 + x + 1$.
 
Блоки оброблюються якимось з режимів шифрування. Було обрано CBC (Cipher Block Chaining), у якому на кожному кроці виконується xor попереднього зашифрованого блоку та поточного ще не зашифрованого блоку, а для першого блоку виконується xor з вектором ініціалізації. Перед шифруванням блоків текст вирівнюється так, щоб його довжина була кратна 128 бітам. Використали алгоритм вирівнювання, у якому додаються байти, які представляють довжину вирівнювання, причому байти додаються навіть якщо початкова довжина була кратна 128 бітам.

Дешуфрування тексту полягає у дешуфруванні 128-бітних блоків відповідно до режиму шифрування. Після чого видаляються біти вирівнювання.

Алгоритм дешуфрування 128-бітного блоку полягає у використанні інверсних операцій у оберненому порядку:
 1. По стовпчиках формується матриця 4x4 з байтів блоку.
 2. $AddRoundKey$ (береться ключ останнього раунду)

    $InvShiftRows$

    $InvSubBytes$
 3. $R-1$ раундів операцій (ключі у зворотному порядку):

    1. $AddRoundKey$

    2. $InvMixColumns$

    3. $InvShiftRows$

    4. $InvSubBytes$
 4. $AddRoundKey$ (використовується ключ 0-го раунду).

 $InvSubBytes$ є застосуванням оберненого до S перетворення, і теж задається константим масивом.

 $InvShiftRows$ є тою самою, що і $ShiftRows$, тільки сдвиг відбувається вправо.

 $InvMixColumns$ є тою самою, що  $MixColumns$, тільки матриця інша.

In [ ]:
def xor_bytes(a, b):
    return bytes(i^j for i, j in zip(a, b))

class AES:
    def __init__(self):
        self.key = None
        self._expanded_key = None
        self.number_of_rounds = 10
        self._s = (
            0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
            0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
            0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
            0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
            0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
            0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
            0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
            0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
            0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
            0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
            0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
            0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
            0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
            0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
            0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
            0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
        )
        self._inv_s = (
            0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
            0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
            0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
            0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
            0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
            0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
            0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
            0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
            0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
            0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
            0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
            0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
            0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
            0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
            0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
            0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D
        )
        self._rc = (
            0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
            0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
            0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
            0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
        )
    def _sub_bytes(self, matrix):
        for i in range(4):
            for j in range(4):
                matrix[i][j] = self._s[matrix[i][j]]

    def _inv_sub_bytes(self, matrix):
        for i in range(4):
            for j in range(4):
                matrix[i][j] = self._inv_s[matrix[i][j]]

    def _shift_rows(self, a):
        a[0][1], a[1][1], a[2][1], a[3][1] = a[1][1], a[2][1], a[3][1], a[0][1]
        a[0][2], a[1][2], a[2][2], a[3][2] = a[2][2], a[3][2], a[0][2], a[1][2]
        a[0][3], a[1][3], a[2][3], a[3][3] = a[3][3], a[0][3], a[1][3], a[2][3]

    def _inv_shift_rows(self, a):
        a[0][1], a[1][1], a[2][1], a[3][1] = a[3][1], a[0][1], a[1][1], a[2][1]
        a[0][2], a[1][2], a[2][2], a[3][2] = a[2][2], a[3][2], a[0][2], a[1][2]
        a[0][3], a[1][3], a[2][3], a[3][3] = a[1][3], a[2][3], a[3][3], a[0][3]

    def _mix_column_multiply(self, a, b):
        p = 0x0;
        for i in range(0, 8):
            if ((b & 0x1) != 0x0):
                p ^= a

            high_bit_set = (a & 0x80) != 0x0
            a <<= 1
            if (high_bit_set):
                a ^= 0x1B
            b >>= 1

        return p

    def _mix_column(self, a):
        a_copy = copy.deepcopy(a)
        a[0] = (self._mix_column_multiply(0x02, a_copy[0]) ^ self._mix_column_multiply(0x03, a_copy[1]) ^ a_copy[2] ^ a_copy[3]) % 256
        a[1] = (a_copy[0] ^ self._mix_column_multiply(0x02, a_copy[1]) ^ self._mix_column_multiply(0x03, a_copy[2]) ^ a_copy[3]) % 256
        a[2] = (a_copy[0] ^ a_copy[1] ^ self._mix_column_multiply(0x02, a_copy[2]) ^ self._mix_column_multiply(0x03, a_copy[3])) % 256
        a[3] = (self._mix_column_multiply(0x03, a_copy[0]) ^ a_copy[1] ^ a_copy[2] ^ self._mix_column_multiply(0x02, a_copy[3])) % 256


    def _mix_columns(self, columns):
        for i in range(4):
            self._mix_column(columns[i])

    def _inv_mix_column(self, a):
        a_copy = copy.deepcopy(a)
        a[0] = (self._mix_column_multiply(14, a_copy[0]) ^ self._mix_column_multiply(11, a_copy[1]) ^ self._mix_column_multiply(13, a_copy[2]) ^ self._mix_column_multiply(9, a_copy[3])) % 256
        a[1] = (self._mix_column_multiply(9, a_copy[0]) ^ self._mix_column_multiply(14, a_copy[1]) ^ self._mix_column_multiply(11, a_copy[2]) ^ self._mix_column_multiply(13, a_copy[3])) % 256
        a[2] = (self._mix_column_multiply(13, a_copy[0]) ^ self._mix_column_multiply(9, a_copy[1]) ^ self._mix_column_multiply(14, a_copy[2]) ^ self._mix_column_multiply(11, a_copy[3])) % 256
        a[3] = (self._mix_column_multiply(11, a_copy[0]) ^ self._mix_column_multiply(13, a_copy[1]) ^ self._mix_column_multiply(9, a_copy[2]) ^ self._mix_column_multiply(14, a_copy[3])) % 256

    def _inv_mix_columns(self, columns):
        for i in range(4):
            self._inv_mix_column(columns[i])

    def _add_round_key(self, matrix, key_matrix):
        for i in range(4):
            for j in range(4):
                matrix[i][j] ^= key_matrix[i][j]

    def _bytes2matrix(self, text_bytes): # list of columns
        return [list(text_bytes[i:i+4]) for i in range(0, len(text_bytes), 4)]

    def _matrix2bytes(self, matrix):
        l = []
        for i in range(4):
            for j in range(4):
                l.append(matrix[i][j])
        return bytes(l)

    def _split_into_128bit_chuncks(self, text_bytes):
        return [text_bytes[i:i+16] for i in range(0, len(text_bytes), 16)]

    def _expand_key(self, key):
        key_columns = self._bytes2matrix(key)
        N = 4
        i = 1
        while len(key_columns) < (self.number_of_rounds + 1) * 4:
            word = list(key_columns[-1])

            if len(key_columns) % N == 0:
                word.append(word.pop(0))
                word = [self._s[b] for b in word]
                word[0] ^= self._rc[i]
                i += 1
            elif N > 6 and len(key_columns) % N == 4:
                word = [self._s[b] for b in word]

            word = xor_bytes(word, key_columns[-N])
            key_columns.append(word)

        self._expanded_key = [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

    def _encrypt_128bit(self, text_bytes):
        matrix = self._bytes2matrix(text_bytes)

        self._add_round_key(matrix, self._expanded_key[0])

        for i in range(1, self.number_of_rounds):
            self._sub_bytes(matrix)
            self._shift_rows(matrix)
            self._mix_columns(matrix)
            self._add_round_key(matrix, self._expanded_key[i])

        self._sub_bytes(matrix)
        self._shift_rows(matrix)
        self._add_round_key(matrix, self._expanded_key[-1])
        
        return self._matrix2bytes(matrix)

    def _decrypt_128bit(self, text_bytes):
        matrix = self._bytes2matrix(text_bytes)

        self._add_round_key(matrix, self._expanded_key[-1])
        self._inv_shift_rows(matrix)
        self._inv_sub_bytes(matrix)

        for i in range(self.number_of_rounds - 1, 0, -1):
            self._add_round_key(matrix, self._expanded_key[i])
            self._inv_mix_columns(matrix)
            self._inv_shift_rows(matrix)
            self._inv_sub_bytes(matrix)

        self._add_round_key(matrix, self._expanded_key[0])

        return self._matrix2bytes(matrix)

    def _pad(self, text_bytes):
        padding_len = 16 - (len(text_bytes) % 16)
        padding = bytes([padding_len] * padding_len)
        return text_bytes + padding
    
    def _unpad(self, text_bytes):
        padding_len = text_bytes[-1]
        message, padding = text_bytes[:-padding_len], text_bytes[-padding_len:]
        return message

    def set_key_with_password(self, password):
        # using my own written md5 hash 
        md5 = Md5()
        self.key = md5.hash(bytes(password, encoding = 'ascii'))
        self._expand_key(self.key)
    
    # CBC mode
    def encrypt(self, text_bytes, iv):  
        if self.key is None:
            print("Key isn't set")
            return None
        text_bytes = self._pad(text_bytes)
        previous = iv
        encrypted_chunks = []
        for chunk in self._split_into_128bit_chuncks(text_bytes):
                encrypted_chunk = self._encrypt_128bit(xor_bytes(chunk, previous))
                encrypted_chunks.append(encrypted_chunk)
                previous = encrypted_chunk
        return b''.join(encrypted_chunks)

    # CBC mode
    def decrypt(self, text_bytes, iv):
        decrypted_chunks = []
        previous = iv
        for chunk in self._split_into_128bit_chuncks(text_bytes):
            decrypted_chunks.append(xor_bytes(previous, self._decrypt_128bit(chunk)))
            previous = chunk

        return self._unpad(b''.join(decrypted_chunks))

In [ ]:
iv = os.urandom(16)

In [ ]:
aes = AES()
aes.set_key_with_password("password")
encrypted = aes.encrypt(b"hello", iv)
print(encrypted.hex())

254bf2cdb487d2d7f044b405193b98c1


In [ ]:
aes.decrypt(encrypted, iv)

b'hello'

Звіримо з бібліотечним AES

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

In [ ]:
key = md5.hash(b"password")
lib_aes = AES.new(key, AES.MODE_CBC, IV=iv)
lib_aes.encrypt(pad(b"hello", AES.block_size)).hex()

'254bf2cdb487d2d7f044b405193b98c1'